# EOmaps Workshop GeoPython 2024
<font size=3>Interactive geo-data analysis with EOmaps and the scientific python infrastructure.</font>

# Data Visualization

EOmaps can be used to visualize both gridded (raster) data or unsorted lists of coordinates and values.  
It provides means to render small datasets as well as millions of datapoints.

> A detailed overview on how to visualize datasets is provided in the [**Data Visualization** section of the EOmaps documentation][docs]

In general, data visualization is split into the following steps:

1. [Assign the data to a `Maps` object with `m.set_data(...)`][assign]
2. [(**optionally**) set the plot-shape with `m.set_shape.< shape >(...)`][shape]
3. [(**optionally**) classify the data with `m.set_classify.< method >(...)`][classify]
4. [Plot the data with `m.plot_map(...)`][plot]

[Maps]: https://eomaps.readthedocs.io/en/latest/generated/eomaps.eomaps.Maps.html
[docs]: https://eomaps.readthedocs.io/en/latest/api_data_visualization.html

[assign]: https://eomaps.readthedocs.io/en/latest/api_data_visualization.html#assign-the-data
[shape]: https://eomaps.readthedocs.io/en/latest/api_data_visualization.html#plot-shapes
[classify]: https://eomaps.readthedocs.io/en/latest/api_data_visualization.html#classify-the-data
[plot]: https://eomaps.readthedocs.io/en/latest/api_data_visualization.html#plot-the-data



## Working with gridded data

In the following we will have a look at how we can use it to visualize some [**Copernicus Soil Water Index**][swi] data!

[xarray][xarray] provides the means to read (and write) most commonly used raster-data formats (NetCDF, GeoTIFF, HDF5, Zarr ...)  

[xarray]: https://docs.xarray.dev/en/stable/index.html
[swi]: https://land.copernicus.eu/en/products/soil-moisture/daily-soil-water-index-global-v3-0-12-5km

In [77]:
# set the backend to use (the default is 'inline')
%matplotlib qt

In [78]:
from pathlib import Path
import xarray as xar

from eomaps import Maps
Maps.config(always_on_top=True)      # keep figures "always on top"

Open one of the NetCDF files:

In [79]:
ncfile = xar.open_dataset(next(Path("gridded_data").iterdir()))
ncfile

<xarray.Dataset> Size: 441MB
Dimensions:    (lat: 1800, lon: 3600, time: 1)
Coordinates:
  * lat        (lat) float64 14kB 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.7 179.8 179.9
  * time       (time) datetime64[ns] 8B 2023-06-01T12:00:00
Data variables: (12/18)
    crs        |S1 1B ...
    SWI_005    (time, lat, lon) float32 26MB ...
    SWI_010    (time, lat, lon) float32 26MB ...
    SWI_001    (time, lat, lon) float32 26MB ...
    SWI_015    (time, lat, lon) float32 26MB ...
    SWI_100    (time, lat, lon) float32 26MB ...
    ...         ...
    QFLAG_100  (time, lat, lon) float32 26MB ...
    QFLAG_020  (time, lat, lon) float32 26MB ...
    QFLAG_005  (time, lat, lon) float32 26MB ...
    QFLAG_001  (time, lat, lon) float32 26MB ...
    QFLAG_060  (time, lat, lon) float32 26MB ...
    SSF        (time, lat, lon) float32 26MB ...
Attributes: (12/25)
    Conventions:          CF-1.6
    archive_facility:     VITO
    copyright:            Copernicus Service information 2023
    geospatial_lat_max:   89.95
    geospatial_lat_min:   -89.95
    geospatial_lon_max:   179.95
    ...                   ...
    region_name:          GLOBE
    sensor:               ASCAT
    source:               Derived from EO microwave scatterometer observations
    time_coverage_end:    2023-06-01T12:00:00Z
    time_coverage_start:  2023-05-31T12:00:00Z
    title:                Daily Soil Water Index 0.1degree: GLOBE 2023-06-01T...

Since the data contains a 3rd dimension ("time"), we select the first entry to obtain a single layer

In [80]:
data = ncfile.isel(time=0)
data

<xarray.Dataset> Size: 441MB
Dimensions:    (lat: 1800, lon: 3600)
Coordinates:
  * lat        (lat) float64 14kB 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.7 179.8 179.9
    time       datetime64[ns] 8B 2023-06-01T12:00:00
Data variables: (12/18)
    crs        |S1 1B ...
    SWI_005    (lat, lon) float32 26MB ...
    SWI_010    (lat, lon) float32 26MB ...
    SWI_001    (lat, lon) float32 26MB ...
    SWI_015    (lat, lon) float32 26MB ...
    SWI_100    (lat, lon) float32 26MB ...
    ...         ...
    QFLAG_100  (lat, lon) float32 26MB ...
    QFLAG_020  (lat, lon) float32 26MB ...
    QFLAG_005  (lat, lon) float32 26MB ...
    QFLAG_001  (lat, lon) float32 26MB ...
    QFLAG_060  (lat, lon) float32 26MB ...
    SSF        (lat, lon) float32 26MB ...
Attributes: (12/25)
    Conventions:          CF-1.6
    archive_facility:     VITO
    copyright:            Copernicus Service information 2023
    geospatial_lat_max:   89.95
    geospatial_lat_min:   -89.95
    geospatial_lon_max:   179.95
    ...                   ...
    region_name:          GLOBE
    sensor:               ASCAT
    source:               Derived from EO microwave scatterometer observations
    time_coverage_end:    2023-06-01T12:00:00Z
    time_coverage_start:  2023-05-31T12:00:00Z
    title:                Daily Soil Water Index 0.1degree: GLOBE 2023-06-01T...

## Visualize the data

In [132]:
m = Maps(layer="all", figsize=(6, 4))
m.add_feature.preset("coastline", "ocean")

### SWI layer
Create a new layer to plot the SWI

In [133]:
m_swi40 = m.new_layer("SWI 40")
m.show_layer(m_swi40.layer)

Plot the "Soil Water Index" with T=40

In [134]:
m_swi40.set_data(data, "lon", "lat", parameter="SWI_040", crs=ncfile.crs.spatial_ref)
m_swi40.set_shape.shade_raster()
m_swi40.plot_map()

Add a colorbar

In [135]:
m_swi40.add_colorbar(hist_bins=50)

### SSF layer
Create a new layer to plot the "Surface State Flag"

In [136]:
m_ssf = m.new_layer("SSF")
m.show_layer(m_ssf.layer)

In [137]:
m_ssf.set_data(data, "lon", "lat", parameter="SSF", crs=ncfile.crs.spatial_ref)
m_ssf.set_shape.shade_raster()
m_ssf.set_classify.UserDefined(bins=[0.5, 1.5, 2.5, 3.5])
m_ssf.plot_map(cmap="RdYlGn_r", vmin=0.5, vmax=3.5)

In [138]:
cb = m_ssf.add_colorbar(hist_bins="bins", hist_kwargs=dict(log=True))
cb.set_bin_labels([0.5, 1.5, 2.5, 3.5], ["unknown", "unfrozen", "frozen", "melting or surface water", "missing"])

## Attach some callbacks

In [141]:
m_swi40.cb.pick.attach.annotate()
m_swi40.cb.pick.set_execute_on_all_layers(True)

m_ssf.cb.pick.attach.annotate(xytext=(0.1, 0.1), textcoords="axes fraction", arrowprops=None, text=lambda val, **kwargs: f"SSF={val}", layer="all")
m_ssf.cb.pick.set_execute_on_all_layers(True)

In [105]:
m.show()

### IPython Widgets

In [142]:
from eomaps import widgets

In [143]:
widgets.LayerOverlaySlider(m, "SSF")

LayerOverlaySlider(value=0.0, description="Overlay\n'SSF':", max=1.0, step=0.01)

In [144]:
display(widgets.LayerButton(m, "SSF"), widgets.LayerButton(m, "SWI 40"))

LayerButton(description='SSF', style=ButtonStyle())

LayerButton(description='SWI 40', style=ButtonStyle())

## Quick plot functions

In [147]:
filepath = next(Path("gridded_data").iterdir())

m_quick = Maps.from_file.NetCDF(filepath, parameter="SWI_001", data_crs=4326, isel=dict(time=0), shape="shade_raster", coastline=True)

# [$\Rightarrow$ Spatio-temporal data analysis](<03 Spatiotemporal_analysis.ipynb>)